In [226]:
import os
import openpyxl 
import collections
import numpy as np
import pandas as pd
from tqdm import tqdm
from difflib import SequenceMatcher

In [227]:
def compare_customer(name1, name2):
    """
    比对客户名称的差异
    :param name1: (str); DDI的客户名称
    :param name2: (str); 商业的客户名称
    """
    conf = SequenceMatcher(None, name1, name2).ratio()
    if conf >= 0.8:
        return True
    if name2 in database.index:
        return name1 == database.loc[name2]["标准名"]
    if conf <= 0.2:
        return False
    new_database[name2] = name1
    return True

def compare_row(r1, r2):
    """
    定义数据比对方法
    :param r1: (Series); DDI流向
    :param r2: (Series); 商业流向
    """
    if r1["批号"] != r2["批号"]:
        return None
    if r1["销售时间"] != r2["销售时间"]:
        return None
    if not compare_customer(r1["客户名称"], r2["客户名称"]):
        return ""
    return r1["数量"] - r2["数量"]


def normalize(value: pd.Series):
    """规范化字符串"""
    value = value.str.strip()
    value = value.str.normalize("NFKC")
    value = value.str.upper()
    return value


In [228]:
ddi_name, bss_name, db_name, file_name = "DDI流向", "商业流向", "别名导入模版", "厦门片仔癀宏仁医药有限公司22年1-12月流向"
file_folder = r"C:\Users\Administrator\Desktop\NewFolder"
file_path = os.path.join(file_folder, f"{file_name}.xlsx")
database_path = os.path.join(file_folder, f"{db_name}.xlsx")
new_database = {}
# 读取第一个Excel表格
ddi_df = pd.read_excel(file_path, sheet_name=ddi_name)
# 读取第二个Excel表格
bss_df = pd.read_excel(file_path, sheet_name=bss_name)
# 读取数据库EXCEL表格
database = pd.read_excel(database_path)
database["别名"] = normalize(database["别名"])
database["标准名"] = normalize(database["标准名"])
database = database.set_index("别名")
if not database.index.is_unique:
    raise Exception("数据库的别名存在重复项")
print(f"完成表1、表2以及数据库的数据读取")

完成表1、表2以及数据库的数据读取


In [229]:
# 定义结果所需的数据
save_column = ["销售时间", "经销商名称", "客户名称", "原始产品名称", "原始产品规格", "产品基本单位", "数量", "批号", "核查结果"]
sort_column = ["销售时间", "批号", "客户名称", "数量", "原始产品名称", "经销商名称", "原始产品规格", "产品基本单位"]

In [230]:
# 剔除无用信息并归一化列名
ddi_df = ddi_df.rename(columns={"原始客户名称": "客户名称", "原始单位": "产品基本单位", "原始数量": "数量"})
ddi_df = ddi_df[save_column[:-1]]
ddi_df = ddi_df.reindex(columns=sort_column)
ddi_df["客户名称"] = normalize(ddi_df["客户名称"])

bss_df = bss_df.rename(columns={"原始客户名称": "客户名称", "原始单位": "产品基本单位", "原始数量": "数量"})
bss_df["经销商名称"] = [""] * len(bss_df)
bss_df = bss_df[save_column[:-1]]
bss_df = bss_df.reindex(columns=sort_column)
bss_df["客户名称"] = normalize(bss_df["客户名称"])


In [231]:
# 对表格进行合并排序，且将数据格式归一化: 销售时间，批文
print("开始第一次差异比对准备")
diff = pd.concat([ddi_df, bss_df])
diff["销售时间"] = pd.to_datetime(diff["销售时间"])
diff["销售时间"] = diff["销售时间"].dt.date
diff["批号"] = diff["批号"].astype(str)
diff["来源"] = [ddi_name]*len(ddi_df) + [bss_name]*len(bss_df)
diff.sort_values(by=[*sort_column[:4], "来源"], inplace=True)

开始第一次差异比对准备


In [232]:
# 第一次过滤大量前后相同的数据
print("开始第一次数据差异比对")
temp_diff = collections.deque()
for index, row in diff.iterrows():
    if row["来源"] == ddi_name or len(temp_diff) == 0:
        temp_diff.append(row)
        continue
    if temp_diff[-1]["来源"] == bss_name:
        temp_diff.append(row)
        continue
    ddi_row = temp_diff.pop()
    if compare_row(ddi_row, row) == 0:
        continue
    temp_diff.append(ddi_row)
    temp_diff.append(row)
diff = pd.DataFrame(temp_diff)
print(f"第一次数据差异比对结束，差异总数:{len(diff)}")

开始第一次数据差异比对
第一次数据差异比对结束，差异总数:736


In [233]:
print("开始合并重复数据")
temp_diff = collections.deque()
diff["合并数量"] = [""] * len(diff)
last_row = diff.iloc[-1]
for _, row in diff.iterrows():
    if last_row["销售时间"] != row["销售时间"] or last_row["批号"] != row["批号"] \
        or last_row["客户名称"] != row["客户名称"] or last_row["来源"] != row["来源"]:
        temp_diff.append(row)
        last_row = row
        continue
    last_row["数量"] += row["数量"]
    if last_row["合并数量"] == "":
        last_row["合并数量"] = f"{last_row.name}-{row.name}"
    else:
        last_row["合并数量"] += f"-{row.name}"
diff = pd.DataFrame(temp_diff)

开始合并重复数据


In [234]:
def complex_compare(row, save_data, control_data):
    if len(control_data) == 0:
        save_data.append(row)
        return
    row_is_del = False
    temp_list = []
    while 1:
        if len(control_data) == 0:
            break
        control_row = control_data.pop()
        res = compare_row(row, control_row) if row["来源"] == ddi_name else compare_row(control_row, row)
        if res == 0:
            row_is_del = True
            break
        temp_list.append(control_row)
        if res is None:
            break
    for control_row in temp_list:
        control_data.append(control_row)
    if not row_is_del:
        save_data.append(row)

In [235]:
print("开始第二次数据差异比对")
ddi_diff = collections.deque()
bss_diff = collections.deque()
for _, row in diff.iterrows():
    if row["来源"] == ddi_name:
        complex_compare(row, ddi_diff, bss_diff)
    else:
        complex_compare(row, bss_diff, ddi_diff)
ddi_diff = pd.DataFrame(ddi_diff)
bss_diff = pd.DataFrame(bss_diff)

开始第二次数据差异比对


In [236]:
len(bss_diff)

1

In [237]:
print("第二次数据差异比对结束，结果如下:")
ddi_diff_num = np.sum(ddi_diff["数量"].tolist()) if len(ddi_diff) != 0 else 0
bss_diff_num = np.sum(bss_diff["数量"].tolist()) if len(bss_diff) != 0 else 0
print(f"{ddi_name}差异条数:{len(ddi_diff)}, 总数量:{ddi_diff_num}")
print(f"{bss_name}差异条数:{len(bss_diff)}, 总数量:{bss_diff_num}")

第二次数据差异比对结束，结果如下:
DDI流向差异条数:0, 总数量:0
商业流向差异条数:1, 总数量:-1


In [238]:
# 保存差异比对结果
print("保存差异比对结果")
df1_save = ddi_diff.copy()
df2_save = bss_diff.copy()
if len(df1_save) != 0:
    del df1_save["来源"]
if len(df2_save) != 0:
    del df2_save["来源"]
df1_save = df1_save.reindex(columns=save_column[:-1])
df2_save = df2_save.reindex(columns=save_column[:-1])
save_path = r"C:\Users\Administrator\Desktop\NewFolder\核查报告.xlsx"
writer = pd.ExcelWriter(save_path)
df1_save.to_excel(writer, index=False, sheet_name=ddi_name)
df2_save.to_excel(writer, index=False, sheet_name=bss_name)
writer.save()
print("保存完成")

保存差异比对结果
保存完成


In [239]:
if len(new_database) == 0:
    print("数据库无新增内容")
else:
    print(f"数据库新增内容,请进行人工核对.新增数量：{len(new_database)}")
    wb = openpyxl.load_workbook(database_path)
    ws = wb['Sheet1']
    for key, value in new_database.items():
        ws.append([key, value, file_name, "未核查"])
    wb.save(database_path)

数据库无新增内容


In [240]:
# index = 0
# for row_name, row in diff.iterrows():
#     if row["销售时间"].strftime("%Y%m%d") == "20220104" and row["原始产品名称"] == "足光散":
#         break
#     index += 1
# print(index)
# diff.iloc[index-5:index+15]

In [241]:
# diff_2 = pd.concat([ddi_diff, bss_diff]).copy()
# diff_2.sort_values(by=[*sort_column, "来源"], inplace=True)

In [242]:
# diff_2.head(20)

,销售时间,批号,客户名称,数量,原始产品名称,经销商名称,原始产品规格,产品基本单位,来源,合并数量
69,2022-01-21,210002,福建耀方药业有限公司,-1,十全大补丸,,9g*10丸,盒,商业流向,
